In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_file_name = 'ETH_USD_data_2019-01-01_to_2021-12-31.csv'

df = pd.read_csv(data_file_name)
df.columns = ['datetime', 'volume', 'open', 'high', 'low', 'close']

df.datetime = pd.to_datetime(df.datetime)

df = df.set_index('datetime')

df.head()

,volume,open,high,low,close
datetime,,,,,
2020-01-02 05:00:10,1,129.87,129.87,129.87,129.87
2020-01-02 05:00:20,1,129.85,129.85,129.85,129.85
2020-01-02 05:00:25,24,129.86,129.86,129.77,129.85
2020-01-02 05:00:35,6,129.86,129.86,129.85,129.85
2020-01-02 05:00:40,8,129.80,129.82,129.79,129.80


In [47]:
df2 = df.iloc[-5000:]

df2 = df2 / df2.max()

inputs = df2.values.flatten().reshape(1, 25000)
inputs.shape

(1, 25000)

In [50]:
np.random.rand(32, 1,25000)

array([[[0.35426622, 0.55228177, 0.96920427, ..., 0.68937627,
         0.65896906, 0.63807989]],

       [[0.42794133, 0.70000931, 0.06600493, ..., 0.30525347,
         0.39320533, 0.62850543]],

       [[0.48770255, 0.90146069, 0.96891446, ..., 0.17199582,
         0.25039126, 0.40454597]],

       ...,

       [[0.99679559, 0.0942115 , 0.95315386, ..., 0.47631089,
         0.92063308, 0.65687163]],

       [[0.50825559, 0.01020492, 0.26245482, ..., 0.78364085,
         0.01214935, 0.83851437]],

       [[0.99593611, 0.97811077, 0.66309253, ..., 0.43274641,
         0.62625669, 0.18150148]]])

In [43]:
len(df2.values.flatten())

25000

In [48]:
inputs

array([[0.17167382, 0.99357041, 0.99346214, ..., 0.99713504, 0.99691388,
        0.99700983]])

In [54]:
num_actions = 11 # 5 levels of buy + 5 levels of sell + do nothing
LEARNING_RATE = 0.00025


import keras
from keras import layers

def create_q_model():

    inputs = layers.Input(shape=(25000, 1)) # temp

    layer1 = layers.Dense(1024, activation='relu')(inputs)
    layer2 = layers.Dense(1024, activation='relu')(layer1)

    action = layers.Dense(num_actions, activation='softmax')(layer2)

    return keras.Model(inputs=inputs, outputs=action)

# optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE, clipnorm=1.0)

# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()

prediction = model.predict(inputs)

print(len(prediction))
print(prediction)

array([[[0.09129392, 0.09026738, 0.09081694, ..., 0.09049427,
         0.09165929, 0.09030642],
        [0.09312288, 0.08722349, 0.09034196, ..., 0.08850001,
         0.09530057, 0.08744205],
        [0.09312265, 0.08722388, 0.09034203, ..., 0.08850028,
         0.09530009, 0.08744243],
        ...,
        [0.09313077, 0.08721039, 0.09033978, ..., 0.08849133,
         0.09531654, 0.08742969],
        [0.09313027, 0.08721118, 0.0903399 , ..., 0.08849186,
         0.09531554, 0.08743046],
        [0.09313049, 0.08721085, 0.09033985, ..., 0.08849163,
         0.09531598, 0.08743013]]], dtype=float32)

In [53]:
model.predict(np.random.rand(32, 1,25000))

ValueError: in user code:

    File "d:\Documents\GitHub\oanda-trading-bot\venv\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "d:\Documents\GitHub\oanda-trading-bot\venv\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Documents\GitHub\oanda-trading-bot\venv\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "d:\Documents\GitHub\oanda-trading-bot\venv\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "d:\Documents\GitHub\oanda-trading-bot\venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Documents\GitHub\oanda-trading-bot\venv\lib\site-packages\keras\e